In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


In [18]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [19]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 32
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [20]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_175_2','COVID-19'],
    subset='training'
) 

Found 350 images belonging to 2 classes.


In [21]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_20_2','COVID-19'],
    subset='training'
)

Found 40 images belonging to 2 classes.


In [22]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal_20_2','COVID-19'],
        #subset='validation'
)

Found 40 images belonging to 2 classes.


In [23]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.DenseNet169(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model.summary()

___________
conv5_block21_0_relu (Activatio (None, 7, 7, 1280)   0           conv5_block21_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_1_conv (Conv2D)   (None, 7, 7, 128)    163840      conv5_block21_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block21_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block21_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block21_1_relu[0][0]       
_________________________________________________________________________________________________

In [24]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
10/10 [==============================] - 50s 4s/step - loss: 1.0078 - accuracy: 0.6805 - precision: 0.7303 - recall: 0.6510 - val_loss: 35.5954 - val_accuracy: 0.5938 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 32s 3s/step - loss: 0.6675 - accuracy: 0.8886 - precision: 0.8986 - recall: 0.8773 - val_loss: 30.9636 - val_accuracy: 0.5000 - val_precision: 1.0000 - val_recall: 0.0588
Epoch 3/5
10/10 [==============================] - 33s 3s/step - loss: 0.3393 - accuracy: 0.9238 - precision: 0.9175 - recall: 0.9341 - val_loss: 2.7114 - val_accuracy: 0.9375 - val_precision: 1.0000 - val_recall: 0.8750
Epoch 4/5
10/10 [==============================] - 33s 3s/step - loss: 0.1546 - accuracy: 0.9555 - precision: 0.9605 - recall: 0.9507 - val_loss: 1.7070 - val_accuracy: 0.9688 - val_precision: 1.0000 - val_recall: 0.9333
Epoch 5/5
10/10 [==============================] - 33s 3s/step - loss: 0.3510 - accuracy: 0.9049 - precisi

In [25]:
model.save('F:/X_ray_Models/Densenet169_xray_model_174_2')

history.history

INFO:tensorflow:Assets written to: F:/X_ray_Models/Densenet169_xray_model_174_2\assets


{'loss': [0.9114088416099548,
  0.5688731074333191,
  0.27531659603118896,
  0.11968731135129929,
  0.34408867359161377],
 'accuracy': [0.7861635088920593,
  0.902515709400177,
  0.9375,
  0.9654088020324707,
  0.9150943160057068],
 'precision': [0.8012422323226929,
  0.9155844449996948,
  0.9320987462997437,
  0.9683544039726257,
  0.9068322777748108],
 'recall': [0.7818182110786438,
  0.8867924809455872,
  0.9437500238418579,
  0.9622641801834106,
  0.9240506291389465],
 'val_loss': [35.595367431640625,
  30.96356773376465,
  2.7114129066467285,
  1.7069997787475586,
  6.409903049468994],
 'val_accuracy': [0.59375, 0.5, 0.9375, 0.96875, 0.6875],
 'val_precision': [0.0, 1.0, 1.0, 1.0, 1.0],
 'val_recall': [0.0,
  0.05882352963089943,
  0.875,
  0.9333333373069763,
  0.4736842215061188]}

In [26]:
results = model.evaluate(test_gen, batch_size=10)

2/2 [==============================] - 4s 713ms/step - loss: 3.5998 - accuracy: 0.7750 - precision: 1.0000 - recall: 0.5500


In [27]:
y_pred = model.predict(test_gen, verbose=1)

#y_pred = np.argmax(y_pred, axis = 1)


for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

2/2 [==============================] - 6s 701ms/step


In [28]:
y_true=test_gen.labels

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.52      0.75      0.61        20
           1       0.55      0.30      0.39        20

    accuracy                           0.53        40
   macro avg       0.53      0.53      0.50        40
weighted avg       0.53      0.53      0.50        40



In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[15,  5],
       [14,  6]], dtype=int64)

In [31]:
results = model.evaluate(test_gen, batch_size=128)

2/2 [==============================] - 4s 844ms/step - loss: 3.5998 - accuracy: 0.7750 - precision: 1.0000 - recall: 0.5500
